In [ ]:
import Pkg; 

Pkg.add("CSV") # install CSV
Pkg.add("DataFrames") # install Dataframes
Pkg.add("TimeSeries")
Pkg.add("Pandas")
Pkg.add("Plots") # plotting dataframe
Pkg.add("StatsPlots")
Pkg.add("RollingFunctions") # rolling functions

In [ ]:
# import libraries
using CSV;
using Dates;
using DataFrames;
using Statistics;
using Plots;
using StatsPlots;
using RollingFunctions;

In [ ]:
# load data
df = CSV.File("../../database/hkex_ticks_day/hkex_0001.csv") |> DataFrame

first(df,5)

### 1. Exploratory Data Analysis (EDA)

In [ ]:
first(df, 5) # show first 5 rows

In [ ]:
last(df, 5) # last 5 rows

In [ ]:
describe(df)

In [ ]:
CSV.write("test.csv", df) # save as csv

In [ ]:
names(df) # column names

In [ ]:
show(df, allrows=true) 

In [ ]:
propertynames(df)

In [ ]:
# select row within date range
first(df[(df.Date .> Date(2017, 1)) .& (df.Date .< Date(2019, 1)), :])

In [ ]:
dates = [Date(2017, 1),Date(2018)];
yms = [yearmonth(d) for d in dates];
print(yms)

first(df[in(yms).(yearmonth.(df.Date)), :], 10)

In [ ]:
close = select(df, :Close) # select column
first(close, 5)

In [ ]:
date = select(df, :Date)
first(date, 5)

### 2. Building trading strategy

In [ ]:
signals = df[:,[:Date, :Close]]

first(signals,5)

In [ ]:
short_window = 40
close = convert(Matrix, select(df, :Close)) # get closing price

short_mavg = runmean(vec(close), short_window)
insertcols!(signals, 1, :short_mavg => short_mavg)

first(signals, 5)

In [ ]:
long_window = 100

long_mavg = runmean(vec(close), long_window)
insertcols!(signals, 1, :long_mavg => long_mavg)

first(signals, 5)

In [ ]:
# create signals
signal = Float64[]
        
for i in 1:length(short_mavg)
    if short_mavg[i] > long_mavg[i]
        x = 1.0
    else
        x = 0.0
    end
    
    push!(signal, x)
end

insertcols!(signals, 1, :signal => signal)

first(signals, 5)

In [ ]:
function gen_pos(signal)
    positions = zeros(length(signal))
    positions[1] = 0
    for i in 2:length(signal)
        positions[i] = signal[i] - signal[i-1]
    end
    return positions
end

positions = gen_pos(signal)

insertcols!(signals, 1, :positions => positions)

first(signals, 5)

In [ ]:
#mycolours = [:green :orange]
#plot(:Date, :short_mavg, group=:ma, linestyle = :solid, linewidth=1, label = "short_mavg", color=mycolours)

df = DataFrame(
  fruit       = ["orange","orange","orange","orange","apple","apple","apple","apple"],
  year        = [2010,2011,2012,2013,2010,2011,2012,2013],
  production  = [120,150,170,160,100,130,165,158],
  consumption = [70,90,100,95,   80,95,110,120]
)

pyplot()
mycolours = [:green :orange] # note that the serie is piled up alphabetically
@df df plot(:year, :production, group=:fruit, linestyle = :solid, linewidth=3, label = reshape(("Production of " .* sort(unique(:fruit))),(1,:)), color=mycolours)
@df df plot!(:year, :consumption, group=:fruit, linestyle = :dot, linewidth=3, label = reshape(("Consumption of " .* sort(unique(:fruit))),(1,:)), color=mycolours)

In [ ]:
pyplot()

@df signals plot(:Date, :short_mavg, linestyle = :solid, linewidth=1, label="Short MA", color="blue")
@df signals plot!(:Date, :long_mavg, linestyle = :solid, linewidth=1, label="Long MA", color="orange")
#@df signals plot!(:Date, :Close, linestyle = :solid, linewidth=0.5, label="Closing price", color="red")

ys = Vector[rand(10), rand(20)]
@df signals plot!(:Date, :short_mavg, color = [:black :orange], line = (:dot, 1), marker = ([:hex :d], 1, 0.8, Plots.stroke(1, :gray)))

In [ ]:
# plot buy signal
signals[(signals[:positions] .== 1),:]